<a href="https://colab.research.google.com/github/charan6366/Lets-Upgrade-AI-ML/blob/master/Twitter_Sentiment_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [2]:
consumer_key = "LzsCOl12ZZ79FObx7BV9a1R3a"
consumer_secret = "gODxZ1kHJHDGgYfyP12AUN37UTlDd6yn3OsSV9Tl0iwu0K0cRY"
access_token = "296102538-kYIK8SULCmRK4vgvYD2DhKbwvSXrgTdTkWh0Nvsp"
access_token_secret = "Ib9obwgcE5tZk4S6tY5rAqI5iXOeJc9FGFg28qHoa4YpP"

In [3]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [4]:
query = "RCB winning IPL"

In [5]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x7fb63aa71a58>, _json={'created_at': 'Thu Oct 29 03:35:51 +0000 2020', 'id': 1321656970986758144, 'id_str': '1321656970986758144', 'full_text': '@SimplyWazza Bhai most of these rcb fans are frustrated that kohli cant get them bragging rights of winning IPL. Ab kohli ko gaali nahi de sakte so they blame everything associated with him.', 'truncated': False, 'display_text_range': [13, 190], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'SimplyWazza', 'name': 'VINAY', 'id': 207561391, 'id_str': '207561391', 'indices': [0, 12]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': 1321656409881128960, 'in_reply_to_status_id_str': '1321656409881128960', 'in_reply_to_user_id': 207561391, 'in_reply_to_user_id_str': '207561391', 'in_reply_to_screen_name': 'SimplyWazza', 'user':

In [6]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [7]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [ ]:
stream(data=["RCB winning IPL"] , file_name='my_tweets')

In [9]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,@SimplyWazza Bhai most of these rcb fans are f...,Desi Gooner,171338,6756,India,False,0,0,2020-10-29 03:35:51
1,RT @IndiaToday: India head coach #RaviShastri ...,Imran,32698,512,India O India Kya Kehna Tera..,False,0,2,2020-10-29 03:16:42
2,@imVkohli @RCBTweets We are still rcb fans eve...,PS Star 09,3,0,,False,0,0,2020-10-29 03:06:55
3,RT @IndiaToday: India head coach #RaviShastri ...,IndiaTodayFLASH,390584,1121307,,True,0,2,2020-10-29 03:06:02
4,India head coach #RaviShastri has urged #Surya...,IndiaToday,791788,5397976,India,True,27,2,2020-10-29 03:05:39


In [10]:
from textblob import TextBlob

In [11]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [12]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [13]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [14]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,@SimplyWazza Bhai most of these rcb fans are f...,Desi Gooner,171338,6756,India,False,0,0,2020-10-29 03:35:51,Bhai most of these rcb fans are frustrated tha...
1,RT @IndiaToday: India head coach #RaviShastri ...,Imran,32698,512,India O India Kya Kehna Tera..,False,0,2,2020-10-29 03:16:42,RT India head coach RaviShastri has urged Sury...
2,@imVkohli @RCBTweets We are still rcb fans eve...,PS Star 09,3,0,,False,0,0,2020-10-29 03:06:55,We are still rcb fans even rcb looses RCB Play...
3,RT @IndiaToday: India head coach #RaviShastri ...,IndiaTodayFLASH,390584,1121307,,True,0,2,2020-10-29 03:06:02,RT India head coach RaviShastri has urged Sury...
4,India head coach #RaviShastri has urged #Surya...,IndiaToday,791788,5397976,India,True,27,2,2020-10-29 03:05:39,India head coach RaviShastri has urged Suryaku...


In [15]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,@SimplyWazza Bhai most of these rcb fans are f...,Desi Gooner,171338,6756,India,False,0,0,2020-10-29 03:35:51,Bhai most of these rcb fans are frustrated tha...,Positive
1,RT @IndiaToday: India head coach #RaviShastri ...,Imran,32698,512,India O India Kya Kehna Tera..,False,0,2,2020-10-29 03:16:42,RT India head coach RaviShastri has urged Sury...,Positive
2,@imVkohli @RCBTweets We are still rcb fans eve...,PS Star 09,3,0,,False,0,0,2020-10-29 03:06:55,We are still rcb fans even rcb looses RCB Play...,Positive
3,RT @IndiaToday: India head coach #RaviShastri ...,IndiaTodayFLASH,390584,1121307,,True,0,2,2020-10-29 03:06:02,RT India head coach RaviShastri has urged Sury...,Positive
4,India head coach #RaviShastri has urged #Surya...,IndiaToday,791788,5397976,India,True,27,2,2020-10-29 03:05:39,India head coach RaviShastri has urged Suryaku...,Positive


In [16]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @IndiaToday: India head coach #RaviShastri has urged #SuryakumarYadav to remain strong and patient after the Mumbai Indians star hit a m…

Clean tweet:
 RT India head coach RaviShastri has urged SuryakumarYadav to remain strong and patient after the Mumbai Indians star hit a m

Sentiment of the tweet:
 Positive


In [17]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @CricketNDTV: Social media was awash with accolades for #SuryakumarYadav whose unbeaten 79, two days after being omitted from the #India…

Clean tweet:
 RT Social media was awash with accolades for SuryakumarYadav whose unbeaten 79 two days after being omitted from the India

Sentiment of the tweet:
 Positive


In [18]:
n = 203
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @Trendulkar: Rohit Sharma is not in the Australia tour squad. So he is out of the IPL too then? So RCB is winning the final against MI?…

Clean tweet:
 RT Rohit Sharma is not in the Australia tour squad So he is out of the IPL too then So RCB is winning the final against MI

Sentiment of the tweet:
 Positive


In [19]:
df[df.Sentiment == 'Positive'].shape[0]

561

In [20]:
df[df.Sentiment == 'Neutral'].shape[0]

45

In [21]:
df[df.Sentiment == 'Negative'].shape[0]

40